In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import psycopg2
from sqlalchemy import create_engine

def create_db(host, user, pwd, db_name, if_exists="fail"):
    con = psycopg2.connect(f"postgresql://{user}:{pwd}@{host}:5432/postgres")
    con.autocommit = True
    cur = con.cursor()
    cur.execute(f"SELECT 1 FROM pg_catalog.pg_database WHERE datname = '{db_name}'")
    exists = cur.fetchone()
    if exists and if_exists == "fail":
        raise Exception(f"Database {db_name} already exists")
    elif exists and if_exists == "replace":
        cur.execute(f"DROP DATABASE {db_name}")
    cur.execute(f"CREATE DATABASE {db_name}")
    con.close()
    print(f"created database {db_name}")

def connect_db(host, user, pwd, db_name):
    engine = create_engine(f"postgresql://{user}:{pwd}@{host}:5432/{db_name}")
    return engine

In [ ]:
host = "XXX.rds.amazonaws.com"
user = "XXX"
pwd = "XXX"
db_name = "berka"

In [ ]:
create_db(host, user, pwd, db_name, if_exists="replace")

## Write Original Data to Database

In [ ]:
engine = connect_db(host, user, pwd, db_name)

In [ ]:
csv_files = list(Path("data").glob("*.csv"))
for fn in csv_files:
    # read data from CSV into Pandas DataFrame
    df = pd.read_csv(fn, sep=';', low_memory=False)
    # ensure all columns are NULL-able
    df = df.convert_dtypes()
    # convert date columns
    cols = [c for c in df.columns if c in ['date', 'issued']]
    for col in cols:
        df[col] = pd.to_datetime(df[col].astype('str').str[:6], format='%y%m%d')
    # get filename w/o extension
    db_table = fn.stem  
    # write DataFrame to DB
    df.to_sql(db_table, engine, index=False, if_exists="replace")
    print(f"created table {db_table} with {df.shape[0]:,} records")

In [ ]:
engine.execute('ALTER TABLE "client" ADD PRIMARY KEY (client_id);')
engine.execute('ALTER TABLE "account" ADD PRIMARY KEY (account_id);')
engine.execute('ALTER TABLE "card" ADD PRIMARY KEY (card_id);')
engine.execute('ALTER TABLE "disp" ADD PRIMARY KEY (disp_id);')
engine.execute('ALTER TABLE "district" ADD PRIMARY KEY (district_id);')
engine.execute('ALTER TABLE "loan" ADD PRIMARY KEY (loan_id);')
engine.execute('ALTER TABLE "order" ADD PRIMARY KEY (order_id);')
engine.execute('ALTER TABLE "trans" ADD PRIMARY KEY (trans_id);')
print(f"created primary keys for all tables")

In [ ]:
engine.execute('ALTER TABLE "account" ADD CONSTRAINT fk_district FOREIGN KEY (district_id) REFERENCES district (district_id);')
engine.execute('ALTER TABLE "client" ADD CONSTRAINT fk_district FOREIGN KEY (district_id) REFERENCES district (district_id);')

engine.execute('ALTER TABLE "disp" ADD CONSTRAINT fk_client FOREIGN KEY (client_id) REFERENCES client (client_id);')
engine.execute('ALTER TABLE "disp" ADD CONSTRAINT fk_account FOREIGN KEY (account_id) REFERENCES account (account_id);')

engine.execute('ALTER TABLE "card" ADD CONSTRAINT fk_disp FOREIGN KEY (disp_id) REFERENCES disp (disp_id);')

engine.execute('ALTER TABLE "trans" ADD CONSTRAINT fk_account FOREIGN KEY (account_id) REFERENCES account (account_id);')
engine.execute('ALTER TABLE "loan" ADD CONSTRAINT fk_account FOREIGN KEY (account_id) REFERENCES account (account_id);')
engine.execute('ALTER TABLE "order" ADD CONSTRAINT fk_account FOREIGN KEY (account_id) REFERENCES account (account_id);')
print(f"created foreign keys for all tables")